# Time Series Analysis

Time series analysis is commonly used to understand trends and seasonality with forecasting as one of the most important applications.

In this tutorial we'll look to analyse trends by:
- Deriving a moving averages using rolling windows
- Making cumulative comparisons by deriving MTD and YTD values

and seasonality by comparing periods like month on month(MoM) or year on year (YoY).

To start we'll need to do some imports and load the [Jupysql](https://jupysql.ploomber.io/en/latest/quick-start.html) extension which allows is to conveniently write SQL directly in cells.

In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt

%load_ext sql

We'll also set some Jupysql configuration options. Other options can be viewed [here](https://jupysql.ploomber.io/en/latest/api/configuration.html).

In [ ]:
# Return the resultset as a pandas dataframe
%config SqlMagic.autopandas = True

# Verbosity level. 0=minimal, 1=normal, 2=all
%config SqlMagic.feedback = 0

# Show connection string after execution
%config SqlMagic.displaycon = False

%config SqlMagic.displaylimit = 10

Connnect to an **in-memory** duckdb database.

In [ ]:
%sql duckdb:///:default:

And now we are ready to do a little data wrangling.

The dataset we'll be using was downloaded from [Kaggle](https://www.kaggle.com/datasets/gabrielsantello/wholesale-and-retail-orders-dataset) and consists of a single csv containing order lines. Let's look at a sample.

In [ ]:
%sql select * from read_csv_auto('./data/retail_orders/orders.csv') limit 5;

It looks like some basic wrangling like renaming the columns, setting the date format and standardising the status column is needed. 

In [ ]:
%%sql

-- Read in the csv file and rename the columns and set the dateformat

create or replace table orders as select * from read_csv_auto(
    './data/retail_orders/orders.csv', 
    names=[
        'customer_id', 
        'status', 
        'order_date', 
        'delivery_date', 
        'order_id', 
        'product_id', 
        'qty', 
        'total', 
        'unit_cost', 
    ],
    skip=1,
    dateformat='%d-%b-%y'
);

-- Standardise the status column as lowercase

update orders set status = lower(status) where lower(status) <> status;

Since most of our analysis will be comparing sales by month let's go ahead and create an aggregate table representing monthly sales.

In [ ]:
%%sql

create or replace table sales_monthly as (
select order_date - (interval (date_part('day', order_date) - 1) day) as sales_month 
,sum(total) as sales
from orders group by 1 order by 1 asc
);

select * from sales_monthly limit 10;

Now for our first bit of trend analysis.

## Rolling Time Windows

A very useful statistic to reveal a trend is to calculate a [moving average](https://en.wikipedia.org/wiki/Moving_average).

In this case we take the average sales over 7 months (current month with 3 months before and after). This is easily achieved using a window function with the very useful preceding and following clauses to give us a rolling window. 

In [ ]:
%%sql --save moving_averages --no-execute

select sales_month 
,sales
,avg(sales)over (order by sales_month rows between 3 preceding and 3 following) as moving_avg
from sales_monthly 
order by 1 asc;

In [ ]:
result = %sql select * from moving_averages;
result.set_index("sales_month").plot()

A cumulative analysis (MTD/YTD) can also be quite useful to understand

In [ ]:
%%sql --save sales --no-execute
with cte as (
select order_date 
,sum(total) as amt 
from orders group by 1 order by 1 asc
)
select 
order_date 
,amt 
,sum(amt) over (partition by year(order_date), month(order_date) order by order_date) as mtd
,sum(amt) over (partition by year(order_date) order by order_date) as ytd
from cte
group by 1, 2 order by 1; 

In [ ]:
result = %sql select * from sales;
result.set_index("order_date").plot()

## Seasonality

### Period comparisons (Mom/YoY)

In [ ]:
%%sql
select sales_month
,sales
,lag(sales_month) over (order by sales_month) as prev_month
,lag(sales) over (order by sales_month) as prev_month_amt
from sales_monthly order by sales_month limit 12;

In [ ]:
%%sql
with cte as (
select sales_month
,sales
,lag(sales, 12) over (order by sales_month) as prev_year_sales
from sales_monthly 
order by sales_month 
)
select * from cte where sales_month >= '2018-01-01'limit 12;

In [ ]:
result = _
result.set_index("order_date").plot()

In [ ]:
%%sql
select date_part('month',order_date) as month_number
,sum(case when date_part('year',order_date) = 2017 then total end) 
 as sales_2017
,sum(case when date_part('year',order_date) = 2018 then total end) 
 as sales_2018
,sum(case when date_part('year',order_date) = 2019 then total end) 
 as sales_2019
,sum(case when date_part('year',order_date) = 2020 then total end) 
 as sales_2020
,sum(case when date_part('year',order_date) = 2021 then total end) 
 as sales_2021
from orders
group by 1;

In [ ]:
result = _
result.set_index("month_number").plot()

## Comparing to Multiple Prior Periods

In [ ]:
%%sql
with cte as (
select sales_month 
,sales
,lag(sales,1) over prev_months as prev_sales_1
,lag(sales,2) over prev_months as prev_sales_2
,lag(sales,3) over prev_months as prev_sales_3
,lag(sales,4) over prev_months as prev_sales_4
from sales_monthly
window prev_months as (
    partition by date_part('month',sales_month) 
    order by sales_month
))
select * from cte where sales_month >= '2021-01-01' order by sales_month asc;

In [ ]:
%%sql
with cte as (
select sales_month 
,sales
,sum(sales) over prev_4_months as sales_prev_4
,sales / sum(sales) over prev_4_months as pct_of_prev_4
from sales_monthly
window prev_4_months as (
    partition by date_part('month',sales_month)
    order by sales_month
    rows between 4 preceding and 1 preceding
))
select * from cte where sales_month >= '2021-01-01' order by sales_month asc;